In [1]:
# Dependencies
import json
import requests 
from pprint import pprint
import pandas as pd
from pandas import json_normalize
import urllib
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
from datetime import datetime, date

In [2]:
# Specify the URL
url = "https://gbfs.citibikenyc.com/gbfs/es/station_status.json"

# Make request and store response
response = requests.get(url)

# Print status code
print(response.status_code)

200


In [3]:
response_json = response.json()
pprint(response_json)

{'data': {'stations': [{'eightd_has_available_keys': False,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1596470524,
                        'legacy_id': '72',
                        'num_bikes_available': 1,
                        'num_bikes_disabled': 2,
                        'num_docks_available': 52,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '72',
                        'station_status': 'active'},
                       {'eightd_has_available_keys': False,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1596469694,
                        'legacy_id': '79',
                        'num_bikes_available': 33,
                        'num_bikes_d

                        'num_bikes_available': 2,
                        'num_bikes_disabled': 1,
                        'num_docks_available': 20,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '270',
                        'station_status': 'active'},
                       {'eightd_has_available_keys': False,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1596470890,
                        'legacy_id': '274',
                        'num_bikes_available': 14,
                        'num_bikes_disabled': 0,
                        'num_docks_available': 17,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '274',
                        'station_status': 'active'},
                   

                       {'eightd_has_available_keys': True,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1596470668,
                        'legacy_id': '304',
                        'num_bikes_available': 22,
                        'num_bikes_disabled': 1,
                        'num_docks_available': 10,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '304',
                        'station_status': 'active'},
                       {'eightd_has_available_keys': False,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1596471116,
                        'legacy_id': '305',
                        'num_bikes_available': 18,
                        'num_bike

                        'num_bikes_available': 34,
                        'num_bikes_disabled': 3,
                        'num_docks_available': 10,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '438',
                        'station_status': 'active'},
                       {'eightd_has_available_keys': False,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1596471119,
                        'legacy_id': '439',
                        'num_bikes_available': 19,
                        'num_bikes_disabled': 0,
                        'num_docks_available': 20,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '439',
                        'station_status': 'active'},
                  

                        'num_docks_available': 24,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '474',
                        'station_status': 'active'},
                       {'eightd_has_available_keys': False,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1596471152,
                        'legacy_id': '476',
                        'num_bikes_available': 7,
                        'num_bikes_disabled': 1,
                        'num_docks_available': 39,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '476',
                        'station_status': 'active'},
                       {'eightd_has_available_keys': False,
                        'is_installed': 1,
                

...

                       {'eightd_has_available_keys': False,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1596470026,
                        'legacy_id': '3637',
                        'num_bikes_available': 7,
                        'num_bikes_disabled': 1,
                        'num_docks_available': 23,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '3637',
                        'station_status': 'active'},
                       {'eightd_has_available_keys': False,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1596470957,
                        'legacy_id': '3638',
                        'num_bikes_available': 5,
                        'num_bi

                        'num_bikes_disabled': 0,
                        'num_docks_available': 22,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '3778',
                        'station_status': 'active'},
                       {'eightd_has_available_keys': False,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1596470750,
                        'legacy_id': '3779',
                        'num_bikes_available': 9,
                        'num_bikes_disabled': 0,
                        'num_docks_available': 15,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '3779',
                        'station_status': 'active'},
                       {'eightd_has_available_keys': False,
       

                        'legacy_id': '3825',
                        'num_bikes_available': 5,
                        'num_bikes_disabled': 0,
                        'num_docks_available': 24,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '3825',
                        'station_status': 'active'},
                       {'eightd_has_available_keys': False,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1596469395,
                        'legacy_id': '3826',
                        'num_bikes_available': 1,
                        'num_bikes_disabled': 0,
                        'num_docks_available': 17,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '3826',
                        '

                        'is_returning': 1,
                        'last_reported': 1596470294,
                        'legacy_id': '3946',
                        'num_bikes_available': 3,
                        'num_bikes_disabled': 0,
                        'num_docks_available': 18,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '3946',
                        'station_status': 'active'},
                       {'eightd_has_available_keys': False,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1596443581,
                        'legacy_id': '3947',
                        'num_bikes_available': 1,
                        'num_bikes_disabled': 0,
                        'num_docks_available': 17,
                        'num_docks_disabled': 0,
                        'n

                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '3996',
                        'station_status': 'active'},
                       {'eightd_has_available_keys': False,
                        'is_installed': 1,
                        'is_renting': 1,
                        'is_returning': 1,
                        'last_reported': 1596468740,
                        'legacy_id': '3997',
                        'num_bikes_available': 11,
                        'num_bikes_disabled': 0,
                        'num_docks_available': 9,
                        'num_docks_disabled': 0,
                        'num_ebikes_available': 0,
                        'station_id': '3997',
                        'station_status': 'active'},
                       {'eightd_has_available_keys': False,
                        'is_installed': 1,
                        'is_renting': 1,
                       

In [4]:
r = requests.get('https://gbfs.citibikenyc.com/gbfs/en/station_status.json')
df = json_normalize(r.json()['data']['stations'])
df

,num_docks_disabled,legacy_id,num_docks_available,num_bikes_available,station_status,num_bikes_disabled,is_installed,is_returning,last_reported,station_id,num_ebikes_available,eightd_has_available_keys,is_renting
0,0,72,52,1,active,2,1,1,1596470524,72,0,False,1
1,0,79,0,33,active,0,1,1,1596469694,79,0,False,1
2,0,82,3,22,active,2,1,1,1596470873,82,0,False,1
3,0,83,23,38,active,1,1,1,1596471155,83,0,False,1
4,0,116,43,6,active,1,1,1,1596471000,116,0,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,0,4041,16,1,active,1,1,1,1596470374,4041,0,False,1
1031,0,4044,33,13,active,3,1,1,1596470997,4044,0,False,1
1032,0,4045,38,1,active,0,1,1,1596470898,4045,0,False,1
1033,0,4058,6,34,active,1,1,1,1596470781,4058,0,False,1


In [5]:
df2 = df.rename(columns={"num_bikes_available":"bikes_available","num_bikes_disabled":"bikes_disabled","num_docks_available":"docks_available","num_docks_disabled":"docks_disabled","last_reported":"time_reported"})

In [6]:
df2

,docks_disabled,legacy_id,docks_available,bikes_available,station_status,bikes_disabled,is_installed,is_returning,time_reported,station_id,num_ebikes_available,eightd_has_available_keys,is_renting
0,0,72,52,1,active,2,1,1,1596470524,72,0,False,1
1,0,79,0,33,active,0,1,1,1596469694,79,0,False,1
2,0,82,3,22,active,2,1,1,1596470873,82,0,False,1
3,0,83,23,38,active,1,1,1,1596471155,83,0,False,1
4,0,116,43,6,active,1,1,1,1596471000,116,0,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,0,4041,16,1,active,1,1,1,1596470374,4041,0,False,1
1031,0,4044,33,13,active,3,1,1,1596470997,4044,0,False,1
1032,0,4045,38,1,active,0,1,1,1596470898,4045,0,False,1
1033,0,4058,6,34,active,1,1,1,1596470781,4058,0,False,1


In [7]:
bike_df = df2[["station_id","bikes_available","bikes_disabled","docks_available","docks_disabled","time_reported"]]

In [8]:
bike_df

,station_id,bikes_available,bikes_disabled,docks_available,docks_disabled,time_reported
0,72,1,2,52,0,1596470524
1,79,33,0,0,0,1596469694
2,82,22,2,3,0,1596470873
3,83,38,1,23,0,1596471155
4,116,6,1,43,0,1596471000
...,...,...,...,...,...,...
1030,4041,1,1,16,0,1596470374
1031,4044,13,3,33,0,1596470997
1032,4045,1,0,38,0,1596470898
1033,4058,34,1,6,0,1596470781


In [9]:

last_reported = datetime.fromtimestamp(bike_df.loc[0,"time_reported"])

In [10]:
last_reported

datetime.datetime(2020, 8, 3, 9, 2, 4)

In [11]:
def epoch_to_readable(xepoch):
    
    return datetime.fromtimestamp(xepoch)

bike_df['time_reported'] = bike_df.apply(lambda x: epoch_to_readable(x['time_reported']), axis = 1 )

<ipython-input-11-5919d25c1233>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bike_df['time_reported'] = bike_df.apply(lambda x: epoch_to_readable(x['time_reported']), axis = 1 )


In [12]:
bike_df

,station_id,bikes_available,bikes_disabled,docks_available,docks_disabled,time_reported
0,72,1,2,52,0,2020-08-03 09:02:04
1,79,33,0,0,0,2020-08-03 08:48:14
2,82,22,2,3,0,2020-08-03 09:07:53
3,83,38,1,23,0,2020-08-03 09:12:35
4,116,6,1,43,0,2020-08-03 09:10:00
...,...,...,...,...,...,...
1030,4041,1,1,16,0,2020-08-03 08:59:34
1031,4044,13,3,33,0,2020-08-03 09:09:57
1032,4045,1,0,38,0,2020-08-03 09:08:18
1033,4058,34,1,6,0,2020-08-03 09:06:21


In [13]:
db_connection_string = "postgres:jaigurudev@localhost:5432/FinalProject"
engine = create_engine(f'postgresql://{db_connection_string}')

In [14]:
engine.table_names()

['bikes']

In [16]:
bike_df.to_sql(name = 'bikes',con = engine , if_exists = 'append',index = False)

In [17]:
pd.read_sql_query('select * from bikes', con=engine).head()

,station_id,bikes_available,bikes_disabled,docks_available,docks_disabled,time_reported
0,72,1,2,52,0,2020-08-03 09:02:04
1,79,33,0,0,0,2020-08-03 08:48:14
2,82,22,2,3,0,2020-08-03 09:07:53
3,83,38,1,23,0,2020-08-03 09:12:35
4,116,6,1,43,0,2020-08-03 09:10:00
